In [ ]:
# TODO: fill this file out, i did a bunch of stuff lazily in the audio file earlier

In [1]:
import numpy as np
# https://github.com/Samarth-Tripathi/IEMOCAP-Emotion-Detection/blob/master/code/python_files/helper.py
def get_transcriptions(path_to_transcriptions):
    f = open(path_to_transcriptions, 'r').read()
    f = np.array(f.split('\n'))
    transcription = {}
    for i in range(len(f) - 1):
        g = f[i]
        i1 = g.find(': ')
        i0 = g.find(' [')
        ind_id = g[:i0]
        ind_ts = g[i1+2:]
        transcription[ind_id] = ind_ts
    return transcription

In [2]:
path_to_transcriptions = '/home/dstratton/IEMOCAP_full_release/Session1/dialog/transcriptions/Ses01F_impro01.txt'

tra = get_transcriptions(path_to_transcriptions)
# i think you can just glob all txt files from here

In [1]:
from torchemotion.datasets.IemocapDataset import *
iemocap_dataset = IemocapDataset('/home/dstratton/IEMOCAP_full_release')

/tmp/ipykernel_85263/587072343.py:2: FutureWarning: Could not cast to float32, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  iemocap_dataset = IemocapDataset('/home/dstratton/IEMOCAP_full_release')


In [ ]:
trans = iemocap_dataset.df['transcription']

In [ ]:
import matplotlib.pyplot as plt
plt.hist(iemocap_dataset.df["end"] - iemocap_dataset.df["start"])
print(sum(iemocap_dataset.df["end"] - iemocap_dataset.df["start"] > 10))
# remove super long clips
iemocap_dataset.df = iemocap_dataset.df[iemocap_dataset.df["end"] - iemocap_dataset.df["start"] < 10]

In [ ]:
import glob
files = glob.glob('/home/dstratton/IEMOCAP_full_release/Session*/dialog/transcriptions/*.txt')
t = get_transcription_list(files[0])
dialog = " ".join(t)

In [2]:
# todo: map labels to be in range [0, 4), since they aren't right now. maybe thats why nll loss is sad
mapping = dict([(y,x) for x,y in enumerate(sorted(set(iemocap_dataset.df["emotion"] )))])
iemocap_dataset.df["emotion"] = iemocap_dataset.df["emotion"].apply(lambda x: mapping[x])

In [3]:
# splitting into sets
import numpy as np, torch
# https://pytorch.org/docs/stable/data.html?highlight=dataset#torch.utils.data.Dataset
# todo: i could do this if i can rand seed it torch.utils.data.random_split(dataset, lengths, generator=<torch._C.Generator object>)
np.random.seed(0)
indices = np.arange(0, len(iemocap_dataset))
np.random.shuffle(indices)
train_indices = indices[:int(len(iemocap_dataset)*.8)]
val_indices = indices[int(len(iemocap_dataset)*.8):int(len(iemocap_dataset)*.9)]
test_indices = indices[int(len(iemocap_dataset)*.9):]

In [4]:
import pandas as pd
import librosa
import datasets

data_dict = {}
for split_name, indices in {"train": train_indices, "val": val_indices, "test": test_indices}.items():
    ind = 0
    df = pd.DataFrame(iemocap_dataset.df["emotion"].iloc[indices]).reset_index()
    # text
    df['text'] = iemocap_dataset.df["transcription"]
    # audio
    # df['audio'] = None
    # for file_name in iemocap_dataset.df["file"].iloc[indices]:
    #     actual_file_name = '/home/dstratton/IEMOCAP_full_release/' + file_name
    #     s, r = librosa.load(actual_file_name, sr=16000)
    #     df['audio'][ind] = {'array': s, 'sampling_rate': r}
    #     ind += 1
    data_dict[split_name] = datasets.Dataset.from_pandas(df)

data = datasets.DatasetDict(data_dict)

In [5]:
# text
import transformers
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-cased")
model = transformers.AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=4)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [6]:
def tokenize_function(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)
data = data.map(tokenize_function)

  0%|          | 0/4424 [00:00<?, ?ex/s]

  0%|          | 0/553 [00:00<?, ?ex/s]

  0%|          | 0/554 [00:00<?, ?ex/s]

In [7]:
def map_vals(example):
    example["labels"] = int(example["emotion"])
    return example
data = data.map(map_vals, remove_columns=['emotion', 'index'])

  0%|          | 0/4424 [00:00<?, ?ex/s]

  0%|          | 0/553 [00:00<?, ?ex/s]

  0%|          | 0/554 [00:00<?, ?ex/s]

In [8]:
import wandb
wandb.login()
%env WANDB_PROJECT=multimodal-emotion-rec

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: derekdstratton (use `wandb login --relogin` to force relogin)


env: WANDB_PROJECT=multimodal-emotion-rec


In [10]:
from transformers import Trainer, TrainingArguments
import numpy as np
from datasets import load_metric
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments("iemocap-text-model",
                                  num_train_epochs=20,
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch",
                                  report_to="wandb",
                                  learning_rate=1e-5,
                                  per_device_train_batch_size=2)
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=data["train"],
    eval_dataset=data["val"]
)

PyTorch: setting up devices


In [11]:
train_results = trainer.train()
wandb.finish()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 4424
  Num Epochs = 20
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 22120
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: wandb version 0.12.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/home/dstratton/miniconda3/envs/InterpretableMultimodal/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 553
  Batch size = 16
Saving model checkpoint to iemocap-text-model/checkpoint-1106
Configuration saved in iemocap-text-model/checkpoint-1106/config.json
Model weights saved in iemocap-text-model/checkpoint-1106/pytorch_model.bin
/home/dstratton/miniconda3/envs/InterpretableMultimodal/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 553
  Batch size = 16
Saving model checkpoint to i

eval/accuracy,▅█▃▁▂▂▃▂▁▂▁▂▂▂▁▁▂▂▂▂
eval/loss,▁▁▁▂▂▃▄▅▆▆▇▇▇▇██████
eval/runtime,▁▅▄▄▃▃▃▄▄▂█▄█▄▄▄▄█▄▄
eval/samples_per_second,█▄▅▅▆▆▆▅▅▇▁▅▁▅▅▅▅▁▅▅
eval/steps_per_second,█▄▅▅▆▆▆▅▅▇▁▅▁▅▅▅▅▁▅▅
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/learning_rate,████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█████▇▇▆▆▄▃▄▃▃▃▂▂▂▃▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁


In [12]:
predictions = trainer.predict(data["test"])

# test_accuracy is correct, predictions for each class, you can take argmax
predictions.metrics

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 554
  Batch size = 16
/home/dstratton/miniconda3/envs/InterpretableMultimodal/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'test_loss': 6.275918483734131,
 'test_accuracy': 0.26173285198555957,
 'test_runtime': 5.8914,
 'test_samples_per_second': 94.036,
 'test_steps_per_second': 5.941}